# Submit Training Job Demo

In [2]:
import fml_manager
import json, time, requests
import os

The DSL and Config file can be presented in JOSN format. 
Submitting a job with JSON defined in line, can use ```submit_job(self,dsl, config)```. 
Note: the parameters are dict, the JSON string have to transform to dict with ```json.loads```

In [3]:
dsl='''
{
    "components" : {
        "secure_add_example_0": {
            "module": "SecureAddExample"
        }
    }
}
'''
config='''
{
  "initiator": {
    "role": "guest",
    "party_id": 10000
  },
  "job_parameters": {
    "work_mode": 1
  },
  "role": {
    "guest": [
      10000
    ],
    "host": [
      10000
    ]
  },
  "role_parameters": {
    "guest": {
      "secure_add_example_0": {
        "seed": [
          123
        ]
      }
    },
    "host": {
      "secure_add_example_0": {
        "seed": [
          321
        ]
      }
    }
  },
  "algorithm_parameters": {
    "secure_add_example_0": {
        "partition": 10,
        "data_num": 1000
    }
  }
}
'''

manager = fml_manager.FMLManager()
response = manager.submit_job(json.loads(dsl),json.loads(config))

In [5]:
manager.prettify(response, True)
stdout = json.loads(response.content)
jobid = stdout["jobId"]

Success!
{
    "data": {
        "board_url": "http://fateboard:8080/index.html#/dashboard?job_id=2020041712255349152813&role=guest&party_id=10000",
        "job_dsl_path": "/data/projects/fate/python/jobs/2020041712255349152813/job_dsl.json",
        "job_runtime_conf_path": "/data/projects/fate/python/jobs/2020041712255349152813/job_runtime_conf.json",
        "logs_directory": "/data/projects/fate/python/logs/2020041712255349152813",
        "model_info": {
            "model_id": "guest-10000#host-10000#model",
            "model_version": "2020041712255349152813"
        }
    },
    "jobId": "2020041712255349152813",
    "retcode": 0,
    "retmsg": "success"
}


Once the job submitted, we can use ```def query_job(self, query_conditions)``` to query the status. the query_condition is a dict, and can add all job's attributes for querying.

In [6]:
query_condition = {
    "job_id":jobid
}
job_status = manager.query_job(query_condition)
manager.prettify(job_status, True)

Success!
{
    "data": [
        {
            "f_create_time": 1587126353502,
            "f_current_steps": null,
            "f_current_tasks": "[\"2020041712255349152813_secure_add_example_0\"]",
            "f_description": "",
            "f_dsl": "{\"components\": {\"secure_add_example_0\": {\"module\": \"SecureAddExample\"}}}",
            "f_elapsed": 17033,
            "f_end_time": 1587126370835,
            "f_initiator_party_id": "10000",
            "f_is_initiator": 1,
            "f_job_id": "2020041712255349152813",
            "f_name": "",
            "f_party_id": "10000",
            "f_progress": 100,
            "f_role": "guest",
            "f_roles": "{\"guest\": [10000], \"host\": [10000]}",
            "f_run_ip": "172.17.0.12:9380",
            "f_runtime_conf": "{\"initiator\": {\"role\": \"guest\", \"party_id\": 10000}, \"job_parameters\": {\"work_mode\": 1, \"model_id\": \"guest-10000#host-10000#model\", \"model_version\": \"2020041712255349152813\"}, \"

<Response [200]>

In [7]:
for i in range(500):
    time.sleep(1)
    job_detail = manager.query_job(query_condition).json()
    final_status = job_detail["data"][0]["f_status"]
    print(final_status)
    
    if final_status == "failed":
        print("Failed")
        break
    if final_status == "success":
        print("Success")
        break

success
Success


We can also fetch the logs of job submitted, and save it to working folder.

In [6]:
response = manager.fetch_job_log(jobid)

{'retcode': 0, 'directory': './job_2020041711064926536612_log', 'retmsg': 'download successfully, please check ./job_2020041711064926536612_log directory, file name is job_2020041711064926536612_log.tar.gz'}
